<a href="https://colab.research.google.com/github/dimitarpg13/langchain_tutorial/blob/main/langchain_tutorial/notebooks/runtime_config/SimpleRuntimeConfig.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with Runtime Configurations


We want to configure our graph when calling it. For example, we want to specify LLM or system prompt to use at runtime _without polluting the graph state with these parameters_.

Adding runtime configuration, invloves a) specifying a schema for the configuration, b) adding the configuration to the nodes or conditional edges and c) passing the configuration to the graph

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain_core langgraph langchain_openai

Below is a simple example which illustrates these steps:

In [ ]:
from langchain_core.runnables import RunnableConfig
from langgraph.graph import END, StateGraph, START
from typing_extensions import TypedDict

1) Specify the config schema

In [ ]:
class ConfigSchema(TypedDict):
    my_runtime_value: str

2) Define a state and a node which accesses the config

In [ ]:
class State(TypedDict):
    my_state_value: str

def node(state: State, config: RunnableConfig):
    if config["configurable"]["my_runtime_value"] == "a":
        return {"my_state_value": 1}
    elif config["configurable"]["my_runtime_value"] == "b":
        return {"my_state_value": 2}
    else:
        raise ValueError("Unknown values.")

3) Build and compile the graph

In [ ]:
builder = Stategraph(State, config_schema=ConfigSchema)
builder.add_node(node)
builder.add_edge(START, "node")
builder.add_edge("node", END)

graph = builder.compile()

4) Pass in configuration at run time and invoke the graph

In [ ]:
print(graph.invoke({}, {"configurable": {"my_runtime_value": "a"}}))
print(graph.invoke({}, {"configurable": {"my_runtime_value": "b"}}))